In [1]:
import csv
import logging
import string
import sqlite3

In [2]:
logging.basicConfig(filename='word_task2.log',encoding = "utf8", level = logging.INFO, format = "%(asctime)s \t %(levelname)s \t %(message)s")

In [36]:
class words:
    
    def __init__(self, fileName, db_Name, table_Name):
        logging.info(f"Object Created for dataset {fileName}!!!")
        
        self.fileName = fileName
        self.db_Name = db_Name
        self.table_Name = table_Name
        
        self.db = sqlite3.connect(db_Name+".db")
        self.cursor = self.db.cursor()
        
        logging.info(f"Database Created for dataset {fileName}!!!")
     
    def __repr__(self):
        return (f"This is the object of file {self.fileName}")
        
    def data_insertion(self):
        """Reads the given data and store it into list and returns it. """
    
        try:
            querry = "SELECT name FROM sqlite_master WHERE type='table'"
            self.cursor.execute(querry)
            list_tablenames = [i[0] for i in self.cursor.fetchall()]
            
            if self.table_Name in list_tablenames:
                self.cursor.execute(f"DROP table {self.table_Name}")
                self.cursor.execute(f"create table {self.table_Name}(col1 text)")
                logging.info(f"Table {self.table_Name} created successfully!!")                   
            else:
                self.cursor.execute(f"create table {self.table_Name}(col1 text)")
                logging.info(f"Table {self.table_Name} created successfully!!")
                
            logging.info(f"Reading Dataset: {self.fileName}")
            with open(self.fileName, "r+", encoding = "utf8") as f:
                data = csv.reader(f, delimiter = "\n")
                for i in data:
                    if i[0][-1]=="_":
                        i[0]=i[0].strip("_")
                    self.cursor.execute("INSERT into "+self.table_Name+" VALUES(?);",i)
                
                self.cursor.execute(f"SELECT * from {self.table_Name}")
                data = self.cursor.fetchall()
                self.db.commit()
            logging.info(f"Reading Dataset {self.fileName} is successful....!!")
            return data
        
        except Exception as e:
            logging.warning("Problem Occured while reading the data...!!")
            logging.exception("Error:", e)
            
    def occurance_counter(self):
        """It counts the occurrances of each word. Returns the list of tuples of word and its count. """
        try:
            logging.info(f"Counting Occurance of each word in the dataset {self.fileName}")
            self.cursor.execute(f"SELECT col1, count(col1) from {self.table_Name} GROUP BY col1")
            final_data = self.cursor.fetchall()
            return final_data
        except Exception as e:
            logging.warning("Problem occurred in occurance_counter() method")
            logging.exception(e)
    
    def starting_occurance_counter(self):
        """It counts the words starting with same alphabet and returns list of tuples of alphabet and its count."""
        logging.info(f"Counting no. of words starting with same alphabet for dataset {self.fileName}")
        try:
            counter = []
            for i in string.ascii_lowercase:
                self.cursor.execute(f"SELECT count(col1) from {self.table_Name} WHERE col1 LIKE '{i}%'")
                count = self.cursor.fetchall()
                counter.append((i,count[0][0]))
            return counter
        except Exception as e:
            logging.warning("Problem occurred in starting_occurance_counter() method")
            logging.exception(e) 
    
    def word_extracter(self,extract_table):
        """This fuction removes all the digits and punctuation returns only words"""
        logging.info(f"Extracting words from the dataset {self.fileName} after removing digits and punctuations")
        
        unwanted  = string.digits + string.punctuation
        
        querry = "SELECT name FROM sqlite_master WHERE type='table'"
        self.cursor.execute(querry)
        list_tablenames = [i[0] for i in self.cursor.fetchall()]
            
        if extract_table in list_tablenames:
            self.cursor.execute(f"DROP table {extract_table}")
            self.cursor.execute(f"create table {extract_table}(col1 text)")                   
        else:
            self.cursor.execute(f"create table {extract_table}(col1 text)")
        
        with open(self.fileName, "r+",encoding = "utf8") as f:
            new_data = csv.reader(f,delimiter = "\n")
            for i in new_data:
                for j in unwanted:
                    i[0] = i[0].replace(j,"")
                if i[0]=="":
                    continue
                else:
                    self.cursor.execute("INSERT into "+extract_table+" VALUES(?);",i)
        self.cursor.execute(f"SELECT * from {extract_table}")
        data2 = self.cursor.fetchall() 
        self.db.commit()
        return data2
    
    def dataset_zipper(self, *args):
        """It returns the list of tuples of all dataset where each records of dataset is mapped to other dataset \n
        to form one single tuple"""
        try:
            logging.info(f"Zipping all {len(args)} datasets into one........")
            zipped = list(zip(*args))
            logging.info("Datasets zipped into one dataset successful")
            return zipped
        except Exception as e:
            logging.warning("Problem occurred while zipping datasets")
            logging.exception(e)
            
    
    def sqlite_database(self,data):
        """This function helps to create in-memory database"""
        logging.info("Creating in-memory database for consolidated datasets")
        
        try: 

            table_name= input("Enter table name: ")
            
            querry = "SELECT name FROM sqlite_master WHERE type='table'"
            self.cursor.execute(querry)
            list_tablenames = [i[0] for i in self.cursor.fetchall()]

            if table_name in list_tablenames:
                self.cursor.execute(f"DROP table {table_name}")
                self.cursor.execute(f"create table {table_name}(col1 text, col2 text, col3 text, col4 text, col5 text)")
                logging.info(f"Table {table_name} created successfully!!!")                   
            else:
                self.cursor.execute(f"create table {table_name}(col1 text, col2 text, col3 text, col4 text, col5 text)")
                logging.info(f"Table {table_name} created successfully!!!")

            for i in data:
                self.cursor.execute("insert into "+table_name+" values(?,?,?,?,?)",i)
            self.cursor.execute(f"select * from {table_name}")
            data = self.cursor.fetchall()
            self.db.commit()
            logging.info("Data inserted into database successfully..!!!!!")
            return data
                
        except Exception as e:
            logging.warning("Problem occured while database creation")
            logging.exception("Error: ",e)
        
    def db_close(self):
        self.db.close()

In [37]:
# Creating object for file 'vocab.enron.txt'
obj1 = words("E:\\Shivansh\\iNeuron\\New folder\\Data\\vocab.enron.txt","bag_of_words","enron")

In [27]:
# Creating object for file 'vocab.kos.txt'
obj2 = words("E:\\Shivansh\\iNeuron\\New folder\\Data\\vocab.kos.txt","bag_of_words","kos")

In [28]:
# Creating object for file 'vocab.nips.txt'
obj3 = words("E:\\Shivansh\\iNeuron\\New folder\\Data\\vocab.nips.txt","bag_of_words","nips")

In [29]:
# Creating object for file 'vocab.nytimes.txt'
obj4 = words("E:\\Shivansh\\iNeuron\\New folder\\Data\\vocab.nytimes.txt","bag_of_words","nytimes")

In [30]:
# Creating object for file 'vocab.pubmed.txt'
obj5 = words("E:\\Shivansh\\iNeuron\\New folder\\Data\\vocab.pubmed.txt","bag_of_words","pubmed")

In [9]:
# Reading data from 'vocab.enron.txt'
enron = obj1.data_insertion()

In [10]:
# Reading data from 'vocab.kos.txt'
kos = obj2.data_insertion()

In [11]:
# Reading data from 'vocab.nips.txt'
nips = obj3.data_insertion()

In [12]:
# Reading data from 'vocab.nytimes.txt'
nytimes = obj4.data_insertion()

In [13]:
# Reading data from 'vocab.pubmed.txt'
pubmed = obj5.data_insertion()

##### Finding occurrrence of each word in each dataset

In [14]:
occur_enron = obj1.occurance_counter()

In [ ]:
for i in occur_enron:
    print(i)

In [15]:
occur_kos = obj2.occurance_counter()

In [ ]:
for i in occur_kos:
    print(i)

In [16]:
occur_nips = obj3.occurance_counter()

In [ ]:
for i in occur_nips:
    print(i)

In [17]:
occur_nytimes = obj4.occurance_counter()

In [ ]:
for i in occur_nytimes:
    print(i)

In [18]:
occur_pubmed = obj5.occurance_counter()

In [ ]:
for i in occur_pubmed:
    print(i)

##### Finding count of each words starting with same alphabet

In [19]:
start_enron = obj1.starting_occurance_counter()

In [ ]:
for i in start_enron:
    print(i)

In [20]:
start_kos = obj2.starting_occurance_counter()

In [ ]:
for i in start_kos:
    print(i)

In [21]:
start_nips = obj3.starting_occurance_counter()

In [ ]:
for i in start_nips:
    print(i)

In [22]:
start_nytimes = obj4.starting_occurance_counter()

In [ ]:
for i in start_nytimes:
    print(i)

In [23]:
start_pubmed = obj5.starting_occurance_counter()

In [ ]:
for i in start_pubmed:
    print(i)

##### Extracting only words after removing all the puctuations and digits from 'vocab.pubmed.txt'

In [31]:
extract_pubmed = obj5.word_extracter("extract_pubmed")

In [ ]:
for i in extract_pubmed:
    print(i[0])

##### Zipping all five datasets into one and creating a in-memory database for it

In [33]:
l1 = [i[0] for i in enron]
l2 = [i[0] for i in kos]
l3 = [i[0] for i in nips]
l4 = [i[0] for i in nytimes]
l5 = [i[0] for i in pubmed]

In [34]:
zipped = obj1.dataset_zipper(l1,l2,l3,l4,l5)

In [35]:
for i in range(0,20):
    print(zipped[i],"\r")

('aaa', 'aarp', 'a2i', 'aah', '>=') 
('aaas', 'abandon', 'aaa', 'aahed', '>>') 
('aactive', 'abandoned', 'aaai', 'aaron', '>>>') 
('aadvantage', 'abandoning', 'aapo', 'aback', '>/=') 
('aaker', 'abb', 'aat', 'abacus', '->') 
('aap', 'abc', 'aazhang', 'abajo', '--') 
('aapg', 'abcs', 'abandonment', 'abalone', '-->') 
('aaron', 'abdullah', 'abbott', 'abandon', '-/-') 
('aarp', 'ability', 'abbreviated', 'abandoned', '-/+') 
('aas', 'aboard', 'abcde', 'abandoning', '/-') 
('aau', 'abortion', 'abe', 'abandonment', '/+-') 
('ab1890', 'abortions', 'abeles', 'abandono', '..') 
('ab1x', 'abraham', 'abi', 'abarnard', '...') 
('ab31x', 'abrams', 'abilistic', 'abashed', '+-') 
('aba', 'abroad', 'abilities', 'abate', '+/') 
('abacus', 'absence', 'ability', 'abated', '+/--') 
('abag', 'absent', 'abl', 'abatement', '+/?') 
('abalone', 'absentee', 'able', 'abating', '+/+') 
('abandon', 'absolute', 'ables', 'abbey', '++') 
('abandoned', 'absolutely', 'ablex', 'abbot', '+++') 


In [38]:
db_data= obj1.sqlite_database(zipped)

Enter table name: zipped_dataset


In [ ]:
for i in db_data:
    print(i)

In [44]:
obj1.cursor.execute("select name from sqlite_master where type='table'")

In [45]:
obj1.cursor.fetchall()

[('enron',),
 ('kos',),
 ('nips',),
 ('nytimes',),
 ('pubmed',),
 ('extract_pubmed',),
 ('zipped_dataset',)]

In [46]:
obj5.db_close()

In [47]:
print(obj1)

This is the object of file E:\Shivansh\iNeuron\New folder\Data\vocab.enron.txt
